# Dynamic attributes

I read the book while on a car trip, so no notes for sections 22.1 and 22.2

# Computed properties

In [ ]:
# V1
import json

JSON_PATH = 'data/osconfeed.json'

class Record:
    def __init__(self, **kwargs) -> None:
        self.__dict__.update(kwargs)

    def __repr__(self) -> str:
        return f'<{self.__class__.__name__} serial={self.serial!r}'

def load(path=JSON_PATH):
    records = {}
    with open(path) as fp:
        raw_data = json.load(fp)
    for collection, raw_records in raw_data['Schedule'].items():
        record_type = collection[:-1] # remove last char, i. e. speakers -> speaker
        for raw_record in raw_records:
            key = f'{record_type}.{raw_record["serial"]}'
            records[key] = Record(**raw_record)

In [ ]:
# V2
import json
import inspect

JSON_PATH = 'data/osconfeed.json'

class Record:
    __index = None

    def __init__(self, **kwargs) -> None:
        self.__dict__.update(kwargs)

    def __repr__(self) -> str:
        return f'<{self.__class__.__name__} serial={self.serial!r}'

    @staticmethod
    def fetch(key):
        if Record.__index is None:
            Record.__index = load()
        return Record.__index[key]
    
class Event(Record):
    def __repr__(self) -> str:
        try:
            return f'<{self.__class__.__name__} {self.name!r}>'
        except AttributeError:
            return super().__repr__()
        
    @property
    def venue(self):
        key = f'venue.{self.venue_serial}'
        return self.__class__.fetch(key)

    @property
    def speakers(self):
        spkr_serials = self.__dict__['speakers']
        fetch = self.__class__.fetch # ?
        return [fetch(f'speker.{key}') for key in spkr_serials]
    

def load(path=JSON_PATH):
    records = {}
    with open(path) as fp:
        raw_data = json.load(fp)
    for collection, raw_records in raw_data['Schedule'].items():
        record_type = collection[:-1] # remove last char, i. e. speakers -> speaker
        cls_name = record_type.capitalize()
        cls = globals().get(cls_name, Record)
        if inspect.isclass(cls) and issubclass(cls, Record):
            factory = cls
        else:
            factory = Record
        for raw_record in raw_records:
            key = f'{record_type}.{raw_record["serial"]}'
            records[key] = factory(**raw_record)

# The code was updated to allow caching, but I will skip it

In [ ]:
# Using property to perform attribute validation

# This code emulates a store that sells organic food (nuts, dry fruits, etc)

class LineItem:
    def __init__(self, description, weight, price) -> None:
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self):
        """This docstring will be shown when doing help(LineItem.weight)"""
        return self.__weight
    
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

In [ ]:
# Property factory (avoid implementing a getter and setter to every quantity)

def quantity(storage_name):
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
        
    return property(qty_getter, qty_setter)

class LineItem:
    weight = quantity('weight')
    # Same code for __init__ and subtotal

In [4]:
# Attributes exclusion

class BlackKnight:
    def __init__(self) -> None:
       self.phrases = [
            ('an arm', "'Tis but a scratch."),
            ('another arm', "It's just a flesh wound."),
            ('a leg', "I'm invincible!"),
            ('another leg', "All right, we'll call it a draw.")
        ]

    @property
    def member(self):
        print('next member is: ') 
        return self.phrases[0][0]
    
    @member.deleter
    def member(self):
        member, text = self.phrases.pop(0)
        print(f'BLACK KNIGHT (loses {member}) -- {text}')

knight = BlackKnight()
print(knight.member)
# next member is: 
# 'an arm'

del knight.member
# BLACK KNIGHT (loses an arm) -- 'Tis but a scratch.

next member is: 
an arm
BLACK KNIGHT (loses an arm) -- 'Tis but a scratch.
